In [ ]:
import tensorflow as tf

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
# flatten helps create the last layer which is the number of outputs that we have
# ImageDataGenerator - helps in image augmentation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/MyDrive/cifar10.zip"


In [ ]:
!unzip -o "/content/drive/MyDrive/cifar10.zip" -d /content/


In [12]:
image_size=[224,224]

In [ ]:
import os

train_path = '/content/cifar10/train'
valid_path = '/content/cifar10/test'

print("Train path exists:", os.path.exists(train_path))
print("Train path contents:", os.listdir(train_path) if os.path.exists(train_path) else "Not found")



In [ ]:
# Count the number of classes
folders = glob(train_path + '/*')
print("Folders found:", folders)
print("Number of classes:", len(folders))

In [ ]:
vgg=VGG16(input_shape=image_size + [3], weights='imagenet', include_top=False)
# resnet=ResNet50(input_shape=image_size + [3], weights='imagenet', include_top=False)

In [19]:
for layer in vgg.layers:
    layer.trainable = False
# for layer in resnet.layers:
# just change vgg to resnet or any other transfer learning model

In [20]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)


In [21]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)



In [ ]:

# view the structure of the model
model.summary()



In [ ]:
# appending folders as a dense layer with an activation function softmax
# x = Dense(1000, activation='relu')(x)
# prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# prediction
# Will have the number of categories

<KerasTensor shape=(None, 0), dtype=float32, sparse=False, name=keras_tensor_20>

In [ ]:
# create a model object
# model = Model(inputs=vgg.input, outputs=prediction)


In [ ]:
!unzip -l "/content/drive/MyDrive/cifar10.zip" | head -40
# this is to check itself


In [23]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# The following are used for image augmentation :
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('cifar10/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('cifar10/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


In [ ]:
# fit the model
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 671s 422ms/step - accuracy: 0.5324 - loss: 1.3787 - val_accuracy: 0.6289 - val_loss: 1.2118
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 682s 436ms/step - accuracy: 0.6688 - loss: 0.9841 - val_accuracy: 0.6769 - val_loss: 0.9306
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 701s 410ms/step - accuracy: 0.6869 - loss: 0.9269 - val_accuracy: 0.6323 - val_loss: 1.1555
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.6960 - loss: 0.9121

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')


In [ ]:
# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')  #converting the model into h5 file